In [ ]:
%load_ext lab_black

In [ ]:
import ohawf
import pandas as pd
from math import isnan
from collections import namedtuple
from apiclient.discovery import build
from datetime import datetime, timedelta
from sqlitedict import SqliteDict as sqldict
from dateutil.relativedelta import relativedelta

In [ ]:
creds = ohawf.get()

In [ ]:
ga_service = build("analytics", "v3", credentials=creds)
get = ga_service.data().ga().get

metrics = [
    "users",
    "sessions",
    "bounceRate",
    "avgSessionDuration",
    "pageviewsPerSession",
]

ga_metrics = ",".join([f"ga:{x}" for x in metrics])
ga_metrics

def ga(brand, gaid, segment, start_date, end_date):
    if segment:
        segment = f"gaid::{segment}"
    req = get(
        ids=f"ga:{gaid}",
        start_date=str(start_date),
        end_date=str(end_date),
        metrics=ga_metrics,
        segment=segment,
        filters="ga:country==United States",
        samplingLevel="HIGHER_PRECISION",
    )
    resp = req.execute()
    return resp

In [ ]:
Args = namedtuple("Args", "brand, gaid, segment, start_date, end_date")
df = pd.read_csv("sites.csv")
with sqldict("ga.db") as db:
    for item in df.to_records():
        x, gaid, name, segment = item
        if type(segment) == float and isnan(segment):
            segment = None
        for x in range(12):
            start_date = datetime.now().date().replace(day=1) - relativedelta(months=x)
            end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
            arg = Args(name, gaid, segment, f"{start_date}", f"{end_date}")
            db[str(arg)] = ga(*arg)
            db.commit()

In [ ]:
table = []
with sqldict("ga.db") as db:
    for arg in db:
        earg = eval(arg)
        year, month, day = earg.start_date.split("-")
        result = [month, year] + db[arg]["rows"][0] + [earg.brand]
        table.append(result)

columns = ["month", "year"] + metrics + ["brand"]
df = pd.DataFrame(table, columns=columns)
df.to_csv("ga_datapull.csv", index=False)

In [ ]:
df